# Modeling NHL Game Outcomes

Building binary classification models to predict the winning team

## Overview

Topline goal: predict the winner of a given NHL game

The first purpose of this model is to see if the winner of an NHL game can be predicted with more accuracy than with a naive prediction - choosing the home team to win every time with no other informative input.

In this notebook several modeling techniques will be trained/tested using team game log data from the prior three full NHL regular seasons (19'-20', 20'-21', 21'-22). Games played to date from the current season (22'-23') will then be used to evaluate the model's ability to correctly predict the winning team.

If I can develop a game prediction model that performs above that naive baseline, the next step will be to adjust the benchmark to a semi naive prediction. The semi naive baseline will be based on choosing the 'Vegas' favorite to win everytime. The recent wave of mobile sports betting legalization has led to massive increase in sports wagers placed both in the US and globally. With the increased popularity of sports betting, sportsbooks odds are an increasingly efficient market i.e. the implied probabilties of sportsbook odds are an unbiased estimator of outcomes.

If I can develop a model which is competitive with the betting market, the next step will be to test betting strategies which leverage the predition model to yield postive ROI. Given the 'vig' (i.e. commission) charged by sportsbooks to take wagers, simply beating the market will not be enough to produce a profitable strategy. Instead the model will have to outperform the market by ~5+% to approach profitability.

The excess return required to be profitable in sports betting and generally high log loss scores in betting models, necessitates the application of betting strategies to the model's prediction. Rather than betting equal amounts on each event, I will test strategies that aim to identify and capitilize on over/undervalued sportsbooks odds.

In [2]:
# Standard Packages
import pandas as pd
# from pandas.testing import assert_frame_equal
import numpy as np
import requests
import re
import time
import os
import warnings

# Scraping packages
import requests
import json
from bs4 import BeautifulSoup
import hockey_scraper

# Viz Packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

# Modeling Packages
## Modeling Prep
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold, \
GridSearchCV, RandomizedSearchCV

## SKLearn Data Prep Modules
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, \
PolynomialFeatures, PowerTransformer, Normalizer, MaxAbsScaler

from sklearn.impute import SimpleImputer

## SKLearn Classification Models
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,\
ExtraTreesClassifier, VotingClassifier, StackingRegressor, GradientBoostingClassifier

## SKLearn Pipeline Setup
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## SKLearn Model Optimization
from sklearn.feature_selection import RFE, f_regression

## Boosting
# from xgboost import XGBClassifier

## SKLearn Metrics
### Classification Scoring/Evaluation
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, \
ConfusionMatrixDisplay, log_loss, confusion_matrix, RocCurveDisplay, make_scorer, roc_auc_score

# ## TensorFlow
# #for the Neural Network
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.wrappers import scikit_learn
# from tensorflow.keras.callbacks import EarlyStopping
# from keras.constraints import maxnorm

In [3]:
# Notebook Config
from pprintpp import pprint as pp
%reload_ext pprintpp
from tqdm import tqdm
from io import StringIO

## Suppress Python Warnings (Future, Deprecation)
import warnings

warnings.filterwarnings("ignore", category= FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Suppress Pandas Warnings (SettingWithCopy)
pd.options.mode.chained_assignment = None

## Pandas Display Config
pd.options.display.max_columns = 80
# pd.options.display.width = None

## Display SKLearn estimators as diagrams
from sklearn import set_config
set_config(display= 'diagram')

## Progress bars
from tqdm.notebook import tqdm
pbar = tqdm(..., display=False) # disable by default
# display(pbar.container) ## display progress bar in desired cell

## EDA

### Data Retrieval

Data sources/notes:
- Game log team statistics are retrieved are retrieved from Natural Stat Trick (NST)
- Game log results data is retrieved from the NHL's API
- Only games from the regular season are considered
    - Ensures all teams are sampled equally
    - Avoids potential issues stemming from structural differences in regular and postseason games
- Data is available/retrieved from the perspective of both the home and away teams
- Metrics describing offensive, defensive and goaltending performance are used to model outcomes
- For each team data for the 3 common game strength states is used
    - 5v5 adjusted
    - 5v4 powerplay (PP)
    - 4v5 penalty kill (PK)
- Team stats are in 'rates' form rather than standard total counts

Rates definitions per NST glossary:
- 5v5 Score & Venue Adjusted 
    - Play where both teams have five skaters and a goalie on the ice, with the event counts adjusted for home ice advantage and leading or trailing score effects. This is done using the method created by Micah Blake McCurdy.
- 5 on 4 PP 
    - Play where the team has five skaters and a goalie on the ice versus four skaters and a goalie their opponent due to penalties.
- 4 on 5 PK 
    - Play where the team has four skaters and a goalie on the ice versus five skaters and a goalie their opponent due to penalties.
- Rates
    - TOI is presented as TOI/GP. For and against statistics are presented as the counts per 60 minutes of play.
  
Processing notes:
- Static data from prior season is used for initial development
- Once the model is built, I will aim to transition to a dynamic/automated ETL process

### Data Constants

In [4]:
# team abbreviation dict from hockey scraper package function
# "_descrition": "# All the corresponding tri-codes for team names",

team_dict = {
    'Anaheim Ducks': 'ANA',
    'Arizona Coyotes' : 'ARI',
    'Boston Bruins': 'BOS', 
    'Buffalo Sabres':'BUF',
    'Calgary Flames': 'CGY', 
    'Carolina Hurricanes': 'CAR', 
    'Chicago Blackhawks': 'CHI', 
    'Colorado Avalanche': 'COL',
    'Columbus Blue Jackets': 'CBJ',
    'Dallas Stars': 'DAL',
    'Detroit Red Wings': 'DET',
    'Edmonton Oilers': 'EDM',
    'Florida Panthers': 'FLA',
    'Los Angeles Kings': 'L.A',
    'Minnesota Wild': 'MIN',
    'Montreal Canadiens': 'MTL',
    'Nashville Predators': 'NSH',
    'New Jersey Devils': 'N.J',
    "New York Islanders": 'NYI',
    "New York Rangers": 'NYR',
    'Ottawa Senators': 'OTT',
    'Philadelphia Flyers': 'PHI',
    'Pittsburgh Penguins': 'PIT',
    'San Jose Sharks': 'S.J',
    'Seattle Kraken': 'SEA',
#     'St. Louis Blues': 'STL',
    'St Louis Blues': 'STL',
    'Tampa Bay Lightning': 'T.B',
    'Toronto Maple Leafs': 'TOR',
    'Vancouver Canucks': 'VAN',
    'Vegas Golden Knights':'VGK',
    'Washington Capitals': 'WSH',
    'Winnipeg Jets': 'WPG'
}

### Game States

Game strength state definitions per NST glossary:
- Even Strength (5v5) Score & Venue Adjusted 
    - Play where both teams have five skaters and a goalie on the ice, with the event counts adjusted for home ice advantage and leading or trailing score effects. This is done using the method created by Micah Blake McCurdy.
- 5 on 4 PP 
    - Play where the team has five skaters and a goalie on the ice versus four skaters and a goalie their opponent due to penalties.
- 4 on 5 PK 
    - Play where the team has four skaters and a goalie on the ice versus five skaters and a goalie their opponent due to penalties.

In [5]:
# game state vars
ev = 'sva'
pp = 'pp'
pk = 'pk'

### Team Statistics

Feature definitions per NST glossary:
- TOI:  
    - Total amount of time played
- GF/60:
    - Rate of Goals for that team per 60 minutes of play. GF*60/TOI
- xGF/60:
    - Expected version of above
- GA/60:
    - Rate of Goals against that team per 60 minutes of play. GA*60/TOI
- xGF/60:
    - Expected version of above
- FF%: 
    - Percentage of total Fenwick in games that team played that are for that team. FF*100/(FF+FA)
- GF%:
    - Percentage of total Goals in games that team played that are for that team. GF*100/(GF+GA)Percentage of total Goals in games that team played that are for that team. GF*100/(GF+GA)
- xGF%:
    - Expected version of above. A metric designed to measure the probability of a shot resulting in a goal. Modeled on shots league wide.
- HDCF%:
    - Percentage of total High Danger Scoring Chances in games that team played that are for that team.
- HDSH%:
    - Percentage of High Danger Shots for that team that were Goals. HDGF*100/HDSF
- HDSV%:
    - Percentage of High Danger Shots against that team that were not Goals. 100-(HDGA*100/HDSA)
- SH%:
    - Percentage of Shots for that team that were Goals. GF*100/SF
- SV%: 
    - Percentage of Shots against that team that were not Goals. 100-(GA*100/SA)

In [6]:
# Even strength 5v5 strength state field list
ev_features_list = [
    'Team',
    'FF%',
    'GF%',
    'xGF%',
    'HDCF%',
    'HDSH%',
    'HDSV%',
    'SH%',
    'SV%',
    'Date', # extracted from 'Game' and used to make 'Game_Key'
    'Game_Key',] # descriptor to be dropped after processing
#     'Game', # descriptor to be dropped after processing
#     'Team', # descriptor to be dropped after processing
#    'TOI', ## don't think i need this

In [7]:
# PP df features list
pp_features_list = [
    'GF/60', 
    'xGF/60', 
    'Game_Key']

In [8]:
# PK df features list
# same as pp df but swap GFs for GAs
pk_features_list = [
    'GA/60', 
    'xGA/60', 
    'Game_Key']
#     'TOI', 

In [9]:
# # Dynamic data loading via NST API
# def get_season_stats_df(season_start_id, season_end_id, strength_state):
#     # season_id is the concatenated season start and end years ex 20222023
#     # strength states defined above - ev, pp, pk
#     url = 'https://www.naturalstattrick.com/games.php?\
#     fromseason={}&thruseason={}&stype=2&sit={}&loc=B&team=All&rate=y'
#     .format(season_start_id, season_end_id, strenth_state)

#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')

#     table = soup.find_all("table")[0]
#     df = pd.read_html(str(table))[0]
#     return df

In [10]:
# # get season stats for 18-19', 19-20', 20-21, 21-22', 22-23'
# evget_season_stats_df(20182019)
# # 5v5 stats
# ev_1819 = get_season_stats_df(20182019, )
# ev_1920 = pd.DataFrame(scrape_schedule('2019-10-02', '2020-03-11', preseason=False, not_over=False))
# ev_2021 = pd.DataFrame(scrape_schedule('2021-01-13', '2021-05-19', preseason=False, not_over=False))
# ev_2122 = pd.DataFrame(scrape_schedule('2021-10-12', '2022-05-01', preseason=False, not_over=False))
# ev_2223 = pd.DataFrame(scrape_schedule('2022-10-07', '2023-04-14', preseason=False, not_over=False))

# # pp stats



# # pk stats

In [11]:
# stats1920_2223get_season_stats_df(20192020, 20222023)

### Data Cleaning/Preprocessing

In [12]:
# 1st round cleaning of team stat sets
def process_stats(game_log_df):
    # extract date from game column
    game_log_df['Date'] = pd.to_datetime(game_log_df['Game'].str[:11])
    # sub full team names with abbreviated names
    game_log_df.replace(team_dict, inplace=True)
    # create game log index
    game_log_df['Game_Key'] = game_log_df['Team'].astype(str)+'_'+game_log_df['Date'].astype(str)
    game_log_df = game_log_df.replace('-', 0)
    return game_log_df

In [13]:
ev_df = pd.read_csv('data/19_23-5v5.csv')
pp_df = pd.read_csv('data/19_23-5v4.csv')
pk_df = pd.read_csv('data/19_23-4v5.csv')

print(ev_df.shape, pp_df.shape, pk_df.shape)

(9148, 33) (9148, 33) (9148, 33)


In [14]:
ev_df

,Game,Team,Unnamed: 2,TOI,CF/60,CA/60,CF%,FF/60,FA/60,FF%,SF/60,SA/60,SF%,GF/60,GA/60,GF%,xGF/60,xGA/60,xGF%,SCF/60,SCA/60,SCF%,HDCF/60,HDCA/60,HDCF%,HDGF/60,HDGA/60,HDGF%,HDSH%,HDSV%,SH%,SV%,PDO
0,"2019-10-02 - Senators 3, Maple Leafs 5",Ottawa Senators,Limited ReportFull Report,44.133333,54.02,78.00,40.92,43.75,59.15,42.52,29.01,43.23,40.16,4.18,5.16,44.73,2.22,3.44,39.27,19.18,34.03,36.04,8.32,15.74,34.58,4.20,5.12,45.08,75.21,44.01,14.40,88.06,1.025
1,"2019-10-02 - Senators 3, Maple Leafs 5",Toronto Maple Leafs,Limited ReportFull Report,44.133333,78.00,54.02,59.08,59.15,43.75,57.48,43.23,29.01,59.84,5.16,4.18,55.27,3.44,2.22,60.73,34.03,19.18,63.96,15.74,8.32,65.42,5.12,4.20,54.92,55.99,24.79,11.94,85.60,0.975
2,"2019-10-02 - Capitals 3, Blues 2",Washington Capitals,Limited ReportFull Report,50.866667,47.21,33.43,58.55,39.14,27.17,59.03,28.43,20.01,58.69,1.20,1.11,51.81,2.26,1.35,62.52,23.09,13.76,62.66,6.14,5.59,52.34,0.00,1.11,0.00,0.00,80.36,4.21,94.44,0.986
3,"2019-10-02 - Capitals 3, Blues 2",St Louis Blues,Limited ReportFull Report,50.866667,33.43,47.21,41.45,27.17,39.14,40.97,20.01,28.43,41.31,1.11,1.20,48.19,1.35,2.26,37.48,13.76,23.09,37.34,5.59,6.14,47.66,1.11,0.00,100.00,19.64,100.00,5.56,95.79,1.014
4,"2019-10-02 - Canucks 2, Oilers 3",Vancouver Canucks,Limited ReportFull Report,47.066667,67.92,44.76,60.28,48.03,30.38,61.25,30.41,24.10,55.79,2.64,3.58,42.39,1.99,1.71,53.76,31.14,19.81,61.12,9.11,6.04,60.13,1.35,2.40,35.98,34.15,60.71,8.67,85.13,0.938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9143,"2023-04-13 - Golden Knights 3, Kraken 1",Seattle Kraken,Limited ReportFull Report,48.383333,73.26,43.38,62.81,52.06,27.71,65.26,29.05,21.04,58.00,1.15,1.31,46.65,2.95,1.33,68.97,30.71,19.78,60.82,14.68,8.05,64.58,0.00,1.31,0.00,0.00,75.58,3.94,93.77,0.977
9144,"2023-04-14 - Avalanche 4, Predators 3",Colorado Avalanche,Limited ReportFull Report,47.016667,73.02,43.19,62.83,52.28,29.98,63.55,41.07,24.08,63.04,5.44,2.43,69.16,3.16,1.92,62.24,31.82,19.12,62.46,12.73,6.99,64.56,2.76,0.00,100.00,24.52,100.00,13.25,89.92,1.032
9145,"2023-04-14 - Avalanche 4, Predators 3",Nashville Predators,Limited ReportFull Report,47.016667,43.19,73.02,37.17,29.98,52.28,36.45,24.08,41.07,36.96,2.43,5.44,30.84,1.92,3.16,37.76,19.12,31.82,37.54,6.99,12.73,35.44,0.00,2.76,0.00,0.00,75.48,10.08,86.75,0.968
9146,"2023-04-14 - Sabres 5, Blue Jackets 2",Buffalo Sabres,Limited ReportFull Report,56.000000,72.69,50.04,59.23,55.55,39.11,58.68,39.23,29.94,56.72,5.65,2.03,73.55,4.32,3.13,58.00,50.99,33.53,60.33,16.15,12.55,56.28,2.29,1.01,69.45,22.47,91.31,14.39,93.22,1.076


In [15]:
# process stats
process_stats(ev_df)
process_stats(pp_df)
process_stats(pk_df)

,Game,Team,Unnamed: 2,TOI,CF/60,CA/60,CF%,FF/60,FA/60,FF%,SF/60,SA/60,SF%,GF/60,GA/60,GF%,xGF/60,xGA/60,xGF%,SCF/60,SCA/60,SCF%,HDCF/60,HDCA/60,HDCF%,HDGF/60,HDGA/60,HDGF%,HDSH%,HDSV%,SH%,SV%,PDO,Date,Game_Key
0,"2019-10-02 - Senators 3, Maple Leafs 5",OTT,Limited ReportFull Report,8.166667,14.69,146.94,9.09,14.69,124.9,10.53,14.69,73.47,16.67,0,7.35,0.00,1.86,7.47,19.93,7.35,66.12,10.00,7.35,7.35,50.00,0,0,0,0.00,100.00,0.00,90.00,0.900,2019-10-02,OTT_2019-10-02
1,"2019-10-02 - Senators 3, Maple Leafs 5",TOR,Limited ReportFull Report,6.000000,10,30,25.00,10,20,33.33,0,0,0,0,0,0,0.24,2.77,7.94,0,20,0.00,0,20,0.00,0,0,0,0,0,0,0,0,2019-10-02,TOR_2019-10-02
2,"2019-10-02 - Capitals 3, Blues 2",STL,Limited ReportFull Report,5.883333,30.59,132.58,18.75,30.59,81.59,27.27,30.59,50.99,37.50,0,10.2,0.00,0.87,7.62,10.23,0,40.79,0.00,0,30.59,0.00,0,0,0,0,100.00,0.00,80.00,0.800,2019-10-02,STL_2019-10-02
3,"2019-10-02 - Capitals 3, Blues 2",WSH,Limited ReportFull Report,3.250000,18.46,92.31,16.67,18.46,73.85,20.00,18.46,36.92,33.33,0,18.46,0.00,0.67,6.44,9.38,0,18.46,0.00,0,18.46,0.00,0,0,0,0,0,0.00,50.00,0.500,2019-10-02,WSH_2019-10-02
4,"2019-10-02 - Canucks 2, Oilers 3",EDM,Limited ReportFull Report,8.000000,7.5,90,7.69,0,82.5,0.00,0,52.5,0.00,0,0,0,0,9.09,0.00,7.5,60,11.11,0,37.5,0.00,0,0,0,0,100.00,0,100.00,0,2019-10-02,EDM_2019-10-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9143,"2023-04-13 - Golden Knights 3, Kraken 1",VGK,Limited ReportFull Report,6.133333,0,136.96,0.00,0,68.48,0.00,0,48.91,0.00,0,0,0,0,5.03,0.00,0,68.48,0.00,0,0,0,0,0,0,0,0,0,100.00,0,2023-04-13,VGK_2023-04-13
9144,"2023-04-14 - Avalanche 4, Predators 3",COL,Limited ReportFull Report,3.300000,18.18,36.36,33.33,18.18,18.18,50.00,18.18,18.18,50.00,0,18.18,0.00,1.16,1.93,37.42,18.18,18.18,50.00,0,18.18,0.00,0,18.18,0.00,0,0.00,0.00,0.00,0.000,2023-04-14,COL_2023-04-14
9145,"2023-04-14 - Avalanche 4, Predators 3",NSH,Limited ReportFull Report,8.000000,0,105,0.00,0,60,0.00,0,22.5,0.00,0,0,0,0,5.99,0.00,0,67.5,0.00,0,22.5,0.00,0,0,0,0,100.00,0,100.00,0,2023-04-14,NSH_2023-04-14
9146,"2023-04-14 - Sabres 5, Blue Jackets 2",BUF,Limited ReportFull Report,2.000000,0,90,0.00,0,60,0.00,0,60,0.00,0,0,0,0,2.11,0.00,0,30,0.00,0,0,0,0,0,0,0,0,0,100.00,0,2023-04-14,BUF_2023-04-14


In [16]:
ev_df = ev_df.replace('-', 0)
pp_df = pp_df.replace('-', 0)
pk_df = pk_df.replace('-', 0)

In [17]:
ev_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9148 entries, 0 to 9147
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Game        9148 non-null   object        
 1   Team        9148 non-null   object        
 2   Unnamed: 2  9148 non-null   object        
 3   TOI         9148 non-null   float64       
 4   CF/60       9148 non-null   float64       
 5   CA/60       9148 non-null   float64       
 6   CF%         9148 non-null   float64       
 7   FF/60       9148 non-null   float64       
 8   FA/60       9148 non-null   float64       
 9   FF%         9148 non-null   float64       
 10  SF/60       9148 non-null   float64       
 11  SA/60       9148 non-null   float64       
 12  SF%         9148 non-null   float64       
 13  GF/60       9148 non-null   float64       
 14  GA/60       9148 non-null   float64       
 15  GF%         9148 non-null   object        
 16  xGF/60      9148 non-nul

In [18]:
ev_df = ev_df[ev_features_list]
pp_df = pp_df[pp_features_list]
pk_df = pk_df[pk_features_list]

In [19]:
# merge team stats (5v5_adj, 5v4_pp, 4v5_pk game states) for each period with the feature column list defined above
def merge_strength_states(ev_df, pp_df, pk_df):
    # add suffix to all columns in ev_df, pp_df, and pk_df
    pp_df = pp_df.rename(columns={col: col + '_pp' if col != 'Game_Key' else col for col in pp_df.columns})
    pk_df = pk_df.rename(columns={col: col + '_pk' if col != 'Game_Key' else col for col in pk_df.columns})
    
    
    # left merge the 5v5 and 5v4 dfs on 'Game_key', w/ respective features, add suffixes for shared cols 
    even_pp_merged = pd.merge(ev_df, pp_df,
                              on = 'Game_Key', how = 'left', suffixes=('', '_pp'))

    # left merge that df with the 4v5_pk on 'Game_key', selected columns, and suffixes for overlapping columns
    all_states_merged = pd.merge(even_pp_merged, pk_df, 
                                  on = 'Game_Key', how = 'left', suffixes = ('', '_pk'))

    return all_states_merged


In [20]:
def convert_to_float(df):
    return df.apply(lambda x: pd.to_numeric(x, errors='ignore') if x.name not in ['Game', 'Team', 'Unnamed: 2','Game_Key', 'Date'] else x)


In [21]:
all_stats_df = merge_strength_states(ev_df, pp_df, pk_df)
all_stats_df

,Team,FF%,GF%,xGF%,HDCF%,HDSH%,HDSV%,SH%,SV%,Date,Game_Key,GF/60_pp,xGF/60_pp,GA/60_pk,xGA/60_pk
0,OTT,42.52,44.73,39.27,34.58,75.21,44.01,14.40,88.06,2019-10-02,OTT_2019-10-02,0,2.77,7.35,7.47
1,TOR,57.48,55.27,60.73,65.42,55.99,24.79,11.94,85.60,2019-10-02,TOR_2019-10-02,7.35,7.47,0,2.77
2,WSH,59.03,51.81,62.52,52.34,0.00,80.36,4.21,94.44,2019-10-02,WSH_2019-10-02,10.2,7.62,18.46,6.44
3,STL,40.97,48.19,37.48,47.66,19.64,100.00,5.56,95.79,2019-10-02,STL_2019-10-02,18.46,6.44,10.2,7.62
4,VAN,61.25,42.39,53.76,60.13,34.15,60.71,8.67,85.13,2019-10-02,VAN_2019-10-02,0,9.09,0,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9143,SEA,65.26,46.65,68.97,64.58,0.00,75.58,3.94,93.77,2023-04-13,SEA_2023-04-13,0,5.03,0,0
9144,COL,63.55,69.16,62.24,64.56,24.52,100.00,13.25,89.92,2023-04-14,COL_2023-04-14,0,5.99,18.18,1.93
9145,NSH,36.45,30.84,37.76,35.44,0.00,75.48,10.08,86.75,2023-04-14,NSH_2023-04-14,18.18,1.93,0,5.99
9146,BUF,58.68,73.55,58.00,56.28,22.47,91.31,14.39,93.22,2023-04-14,BUF_2023-04-14,0,1.26,0,2.11


In [22]:
all_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9148 entries, 0 to 9147
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Team       9148 non-null   object        
 1   FF%        9148 non-null   float64       
 2   GF%        9148 non-null   object        
 3   xGF%       9148 non-null   float64       
 4   HDCF%      9148 non-null   float64       
 5   HDSH%      9148 non-null   object        
 6   HDSV%      9148 non-null   object        
 7   SH%        9148 non-null   float64       
 8   SV%        9148 non-null   float64       
 9   Date       9148 non-null   datetime64[ns]
 10  Game_Key   9148 non-null   object        
 11  GF/60_pp   9148 non-null   object        
 12  xGF/60_pp  9148 non-null   object        
 13  GA/60_pk   9148 non-null   object        
 14  xGA/60_pk  9148 non-null   object        
dtypes: datetime64[ns](1), float64(5), object(9)
memory usage: 1.1+ MB


In [23]:
all_stats_df = convert_to_float(all_stats_df)
print(all_stats_df.dtypes)

Team                 object
FF%                 float64
GF%                 float64
xGF%                float64
HDCF%               float64
HDSH%               float64
HDSV%               float64
SH%                 float64
SV%                 float64
Date         datetime64[ns]
Game_Key             object
GF/60_pp            float64
xGF/60_pp           float64
GA/60_pk            float64
xGA/60_pk           float64
dtype: object


In [24]:
def rolling_features(df, rolling_games=10):
#     df['Date'] = df['Date']
#     df['season_id'] = df['season_id']
#     df['_Team_Won'] = df['_Team_Won']
#     df['rolling_avg_pts_pct'] = df.groupby([df['Date'].dt.year, 'Team'])['_pts_pct'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_pts_pct'] = df.groupby([df['Date'].dt.year, 'Team'])['away_pts_pct'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
    df['rolling_avg_FF%'] = df.groupby([df['Date'].dt.year, 'Team'])['FF%'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
    df['rolling_avg_GF%'] = df.groupby([df['Date'].dt.year, 'Team'])['GF%'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
    df['rolling_avg_xGF%'] = df.groupby([df['Date'].dt.year, 'Team'])['xGF%'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
    df['rolling_avg_HDCF%'] = df.groupby([df['Date'].dt.year, 'Team'])['HDCF%'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
    df['rolling_avg_HDSH%'] = df.groupby([df['Date'].dt.year, 'Team'])['HDSH%'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
    df['rolling_avg_HDSV%'] = df.groupby([df['Date'].dt.year, 'Team'])['HDSV%'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
    df['rolling_avg_SH%'] = df.groupby([df['Date'].dt.year, 'Team'])['SH%'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().mean())
    df['rolling_avg_SV%'] = df.groupby([df['Date'].dt.year, 'Team'])['SV%'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().mean())
    df['rolling_avg_GF/60_pp'] = df.groupby([df['Date'].dt.year, 'Team'])['GF/60_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
    df['rolling_avg_xGF/60_pp'] = df.groupby([df['Date'].dt.year, 'Team'])['xGF/60_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
    df['rolling_avg_GA/60_pk'] = df.groupby([df['Date'].dt.year, 'Team'])['GA/60_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
    df['rolling_avg_xGA/60_pk'] = df.groupby([df['Date'].dt.year, 'Team'])['xGA/60_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
#     df['rolling_avg_away_FF%'] = df.groupby([df['Date'].dt.year, 'Team'])['away_FF%'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_GF%'] = df.groupby([df['Date'].dt.year, 'Team'])['away_GF%'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_xGF%'] = df.groupby([df['Date'].dt.year, 'Team'])['away_xGF%'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_HDCF%'] = df.groupby([df['Date'].dt.year, 'Team'])['away_HDCF%'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_HDSH%'] = df.groupby([df['Date'].dt.year, 'Team'])['away_HDSH%'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_HDSV%'] = df.groupby([df['Date'].dt.year, 'Team'])['away_HDSV%'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_SH%'] = df.groupby([df['Date'].dt.year, 'Team'])['away_SH%'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_SV%'] = df.groupby([df['Date'].dt.year, 'Team'])['away_SV%'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_GF/60_pp'] = df.groupby([df['Date'].dt.year, 'Team'])['away_GF/60_pp'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_xGF/60_pp'] = df.groupby([df['Date'].dt.year, 'Team'])['away_xGF/60_pp'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_GA/60_pk'] = df.groupby([df['Date'].dt.year, 'Team'])['away_GA/60_pk'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
#     df['rolling_avg_away_xGA/60_pk'] = df.groupby([df['Date'].dt.year, 'Team'])['away_xGA/60_pk'].rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
    return df

In [25]:
all_stats_df = rolling_features(all_stats_df)
all_stats_df.head(20)

,Team,FF%,GF%,xGF%,HDCF%,HDSH%,HDSV%,SH%,SV%,Date,Game_Key,GF/60_pp,xGF/60_pp,GA/60_pk,xGA/60_pk,rolling_avg_FF%,rolling_avg_GF%,rolling_avg_xGF%,rolling_avg_HDCF%,rolling_avg_HDSH%,rolling_avg_HDSV%,rolling_avg_SH%,rolling_avg_SV%,rolling_avg_GF/60_pp,rolling_avg_xGF/60_pp,rolling_avg_GA/60_pk,rolling_avg_xGA/60_pk
0,OTT,42.52,44.73,39.27,34.58,75.21,44.01,14.40,88.06,2019-10-02,OTT_2019-10-02,0.00,2.77,7.35,7.47,NaN,NaN,NaN,NaN,NaN,NaN,84.709355,917.569677,NaN,NaN,NaN,NaN
1,TOR,57.48,55.27,60.73,65.42,55.99,24.79,11.94,85.60,2019-10-02,TOR_2019-10-02,7.35,7.47,0.00,2.77,NaN,NaN,NaN,NaN,NaN,NaN,83.725313,919.072188,NaN,NaN,NaN,NaN
2,WSH,59.03,51.81,62.52,52.34,0.00,80.36,4.21,94.44,2019-10-02,WSH_2019-10-02,10.20,7.62,18.46,6.44,NaN,NaN,NaN,NaN,NaN,NaN,97.903437,916.051875,NaN,NaN,NaN,NaN
3,STL,40.97,48.19,37.48,47.66,19.64,100.00,5.56,95.79,2019-10-02,STL_2019-10-02,18.46,6.44,10.20,7.62,NaN,NaN,NaN,NaN,NaN,NaN,80.248437,928.922500,NaN,NaN,NaN,NaN
4,VAN,61.25,42.39,53.76,60.13,34.15,60.71,8.67,85.13,2019-10-02,VAN_2019-10-02,0.00,9.09,0.00,4.60,NaN,NaN,NaN,NaN,NaN,NaN,74.133226,922.073226,NaN,NaN,NaN,NaN
5,EDM,38.75,57.61,46.24,39.87,39.29,65.85,14.87,91.33,2019-10-02,EDM_2019-10-02,0.00,4.60,0.00,9.09,NaN,NaN,NaN,NaN,NaN,NaN,77.374545,911.065152,NaN,NaN,NaN,NaN
6,S.J,36.66,33.91,26.72,24.75,0.00,87.94,6.72,91.93,2019-10-02,S.J_2019-10-02,0.00,5.33,13.85,8.96,NaN,NaN,NaN,NaN,NaN,NaN,82.280000,875.965937,NaN,NaN,NaN,NaN
7,VGK,63.34,66.09,73.28,75.25,12.06,100.00,8.07,93.28,2019-10-02,VGK_2019-10-02,13.85,8.96,0.00,5.33,NaN,NaN,NaN,NaN,NaN,NaN,71.322941,914.321765,NaN,NaN,NaN,NaN
8,FLA,58.93,34.36,47.57,44.02,16.98,80.19,3.55,90.06,2019-10-03,FLA_2019-10-03,0.00,4.25,14.17,10.54,NaN,NaN,NaN,NaN,NaN,NaN,94.796333,899.527667,NaN,NaN,NaN,NaN
9,T.B,41.07,65.64,52.43,55.98,19.81,83.02,9.94,96.45,2019-10-03,T.B_2019-10-03,14.17,10.54,0.00,4.25,NaN,NaN,NaN,NaN,NaN,NaN,90.283793,904.236552,NaN,NaN,NaN,NaN


In [26]:
print(all_stats_df.columns)

Index(['Team', 'FF%', 'GF%', 'xGF%', 'HDCF%', 'HDSH%', 'HDSV%', 'SH%', 'SV%',
       'Date', 'Game_Key', 'GF/60_pp', 'xGF/60_pp', 'GA/60_pk', 'xGA/60_pk',
       'rolling_avg_FF%', 'rolling_avg_GF%', 'rolling_avg_xGF%',
       'rolling_avg_HDCF%', 'rolling_avg_HDSH%', 'rolling_avg_HDSV%',
       'rolling_avg_SH%', 'rolling_avg_SV%', 'rolling_avg_GF/60_pp',
       'rolling_avg_xGF/60_pp', 'rolling_avg_GA/60_pk',
       'rolling_avg_xGA/60_pk'],
      dtype='object')


In [27]:
all_stats_features = ['Team','Date', 'Game_Key','rolling_avg_FF%', 'rolling_avg_GF%', 'rolling_avg_xGF%',
       'rolling_avg_HDCF%', 'rolling_avg_HDSH%', 'rolling_avg_HDSV%',
       'rolling_avg_SH%', 'rolling_avg_SV%', 'rolling_avg_GF/60_pp',
       'rolling_avg_xGF/60_pp', 'rolling_avg_GA/60_pk',
       'rolling_avg_xGA/60_pk']

In [28]:
all_stats_df.isna().sum()

Team                        0
FF%                         0
GF%                         0
xGF%                        0
HDCF%                       0
HDSH%                       0
HDSV%                       0
SH%                         0
SV%                         0
Date                        0
Game_Key                    0
GF/60_pp                    0
xGF/60_pp                   0
GA/60_pk                    0
xGA/60_pk                   0
rolling_avg_FF%          1580
rolling_avg_GF%          1580
rolling_avg_xGF%         1580
rolling_avg_HDCF%        1580
rolling_avg_HDSH%        1580
rolling_avg_HDSV%        1580
rolling_avg_SH%             0
rolling_avg_SV%             0
rolling_avg_GF/60_pp     1580
rolling_avg_xGF/60_pp    1580
rolling_avg_GA/60_pk     1580
rolling_avg_xGA/60_pk    1580
dtype: int64

In [29]:
all_stats_df = all_stats_df[all_stats_features]
all_stats_df = all_stats_df.dropna()
all_stats_df

,Team,Date,Game_Key,rolling_avg_FF%,rolling_avg_GF%,rolling_avg_xGF%,rolling_avg_HDCF%,rolling_avg_HDSH%,rolling_avg_HDSV%,rolling_avg_SH%,rolling_avg_SV%,rolling_avg_GF/60_pp,rolling_avg_xGF/60_pp,rolling_avg_GA/60_pk,rolling_avg_xGA/60_pk
263,DAL,2019-10-21,DAL_2019-10-21,45.523,39.467,49.724,52.622,22.836,81.519,74.160645,938.801613,4.295,6.528,6.862,5.051
264,TOR,2019-10-22,TOR_2019-10-22,52.326,52.638,49.215,47.447,31.934,72.085,83.725313,919.072188,9.489,6.177,5.246,5.625
279,WPG,2019-10-22,WPG_2019-10-22,47.409,36.499,44.182,42.251,9.245,80.552,89.867742,936.664839,5.012,5.437,35.874,7.953
280,VGK,2019-10-22,VGK_2019-10-22,50.686,36.519,53.923,54.019,14.110,84.628,71.322941,914.321765,12.791,8.953,3.432,7.019
282,WSH,2019-10-22,WSH_2019-10-22,54.229,49.310,55.439,52.822,13.942,77.652,97.903437,916.051875,55.926,17.439,5.719,4.657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9143,SEA,2023-04-13,SEA_2023-04-13,57.050,54.449,59.066,56.560,16.971,70.043,100.511538,896.874872,10.103,7.819,1.600,4.080
9144,COL,2023-04-14,COL_2023-04-14,55.047,61.475,53.935,49.601,24.524,81.491,98.016053,921.409737,6.581,7.114,10.536,7.698
9145,NSH,2023-04-14,NSH_2023-04-14,40.670,42.353,39.427,41.112,10.384,94.422,82.664737,922.814211,4.298,7.397,5.177,7.059
9146,BUF,2023-04-14,BUF_2023-04-14,49.983,51.548,49.511,47.102,20.448,86.842,92.056154,894.443590,5.203,5.691,6.636,7.356


In [30]:
all_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7568 entries, 263 to 9147
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Team                   7568 non-null   object        
 1   Date                   7568 non-null   datetime64[ns]
 2   Game_Key               7568 non-null   object        
 3   rolling_avg_FF%        7568 non-null   float64       
 4   rolling_avg_GF%        7568 non-null   float64       
 5   rolling_avg_xGF%       7568 non-null   float64       
 6   rolling_avg_HDCF%      7568 non-null   float64       
 7   rolling_avg_HDSH%      7568 non-null   float64       
 8   rolling_avg_HDSV%      7568 non-null   float64       
 9   rolling_avg_SH%        7568 non-null   float64       
 10  rolling_avg_SV%        7568 non-null   float64       
 11  rolling_avg_GF/60_pp   7568 non-null   float64       
 12  rolling_avg_xGF/60_pp  7568 non-null   float64       
 13  r

In [31]:
# def column_dropper(ev_df, ev_features_list, pp_df, pp_features_list, pk_df, pk_features_list):
#     ev_df = ev_df[ev_features_list]
#     pp_df = pp_df[pp_features_list]
#     pk_df = pk_df[pk_features_list]
#     ev_df = ev_df.replace('-', 0)
#     pp_df = pp_df.replace('-', 0)
#     pk_df = pk_df.replace('-', 0)
#     return ev_df, pp_df, pk_df

### Add NHL.com schedule/result df


In [32]:
# functions from the Hockey Scraper API
# modified to retrieve additional info 
"""
This module contains functions to scrape the json schedule for any games or date range
"""

from datetime import datetime, timedelta
import json
import time
import hockey_scraper.utils.shared as shared


# TODO: Currently rescraping page each time since the status of some games may have changed
# (e.g. Scraped on 2020-01-20 and game on 2020-01-21 was not Final...when use old page again will still think not Final)
# Need to find a more elegant way of doing this (Metadata???)
def get_schedule(date_from, date_to):
    """
    Scrapes games in date range
    Ex: https://statsapi.web.nhl.com/api/v1/schedule?startDate=2010-10-03&endDate=2011-06-20
    
    :param date_from: scrape from this date
    :param date_to: scrape until this date
    
    :return: raw json of schedule of date range
    """
    page_info = {
        "url": 'https://statsapi.web.nhl.com/api/v1/schedule?startDate={a}&endDate={b}'.format(a=date_from, b=date_to),
        "name": date_from + "_" + date_to,
        "type": "json_schedule",
        "season": shared.get_season(date_from),
    }

    return json.loads(shared.get_file(page_info, force=True))


def chunk_schedule_calls(from_date, to_date):
    """
    The schedule endpoint sucks when handling a big date range. So instead I call in increments of n days.
    
    :param date_from: scrape from this date
    :param date_to: scrape until this date

    :return: raw json of schedule of date range
    """
    sched = []
    days_per_call = 30

    from_date = datetime.strptime(from_date, "%Y-%m-%d") 
    to_date = datetime.strptime(to_date, "%Y-%m-%d")
    num_days = (to_date - from_date).days + 1  # +1 since difference is looking for total number of days

    for offset in range(0, num_days, days_per_call):
        f_chunk = datetime.strftime(from_date + timedelta(days=offset), "%Y-%m-%d")

        # We need the min bec. if the chunks are evenly sized this prevents us from overshooting the max
        t_chunk = datetime.strftime(from_date + timedelta(days=min(num_days-1, offset+days_per_call-1)), "%Y-%m-%d")

        chunk_sched = get_schedule(f_chunk, t_chunk)
        sched.append(chunk_sched['dates'])

    return sched


def get_dates(games):
    """
    Given a list game_ids it returns the dates for each game.

    We sort all the games and retrieve the schedule from the beginning of the season from the earliest game
    until the end of most recent season.
    
    :param games: list with game_id's ex: 2016020001
    
    :return: list with game_id and corresponding date for all games
    """
    today = datetime.today()

    # Determine oldest and newest game
    games = list(map(str, games))
    games.sort()

    date_from = shared.season_start_bound(games[0][:4])
    year_to = int(games[-1][:4])

    # If the last game is part of the ongoing season then only request the schedule until Today
    # We get strange errors if we don't do it like this
    if year_to == shared.get_season(datetime.strftime(today, "%Y-%m-%d")):
        date_to = '-'.join([str(today.year), str(today.month), str(today.day)])
    else:
        date_to = datetime.strftime(shared.season_end_bound(year_to+1), "%Y-%m-%d")  # Newest game in sample

    # TODO: Assume true is live here -> Workaround
    schedule = scrape_schedule(date_from, date_to, preseason=True, not_over=True)

    # Only return games we want in range
    games_list = []
    for game in schedule:
        if str(game['game_id']) in games:
            games_list.extend([game])
    return games_list


def scrape_schedule(date_from, date_to, preseason=False, not_over=False):
    """
    Calls getSchedule and scrapes the raw schedule Json
    
    :param date_from: scrape from this date
    :param date_to: scrape until this date
    :param preseason: Boolean indicating whether include preseason games (default if False)
    :param not_over: Boolean indicating whether we scrape games not finished. 
                     Means we relax the requirement of checking if the game is over. 
    
    :return: list with all the game id's
    """
    schedule = []
    schedule_json = chunk_schedule_calls(date_from, date_to)

    for chunk in schedule_json:
        for day in chunk:
            for game in day['games']:
                if game['status']['detailedState'] == 'Final' or not_over:
                    game_id = int(str(game['gamePk'])[5:])
                    # add game type logic to filter out none regular season games
                    if (game_id >= 20000 or preseason) and game_id < 40000:
                        schedule.append({
                                 "game_id": game['gamePk'],
                                "game_type": game['gameType'],
                                 "season_id": game['season'],
                                 "date": day['date'], 
                                 "home_score": game['teams']['home'].get("score"),
                                 "away_score": game['teams']['away'].get("score"),
                                #  "start_time": datetime.strptime(game['gameDate'][:-1], "%Y-%m-%dT%H:%M:%S"),
                                #  "venue": game['venue'].get('name'),
                                #  "home_team_id": game['teams']['home']['team']['id'],
                                 "Home_team": shared.get_team(game['teams']['home']['team']['name']),
                                 "home_wins": game['teams']['home'].get("leagueRecord").get("wins"),
                                 "home_losses": game['teams']['home'].get("leagueRecord").get("losses"),
                                 "home_otl": game['teams']['home'].get("leagueRecord").get("ot"),
                                 "away_wins": game['teams']['away'].get("leagueRecord").get("wins"),
                                 "away_losses": game['teams']['away'].get("leagueRecord").get("losses"),
                                 "away_otl": game['teams']['away'].get("leagueRecord").get("ot"), 
                                #  "away_team_id": game['teams']['away']['team']['id'],
                                 "Away_team": shared.get_team(game['teams']['away']['team']['name']),
                                #  "home_score": game['teams']['home'].get("score"),
                                #  "away_score": game['teams']['away'].get("score"),
                                 "status": game["status"]["abstractGameState"]
                        })


    return schedule

NHL regular season date boundaries by season
- 2018-2019: October 3, 2018 – June 12, 2019
- 2019-2020: October 2, 2019 – March 11, 2020
  - Covid caused scheduling issues, so need to make sure only regular season games are factored
- 2020-2021: January 13, 2021 - May 19, 2021
  - same potential issue noted above
- 2021-2022: October 12, 2021 - May 1, 2022
- 2022-2023: October 7, 2022 - April 14, 2023

In [33]:
# run modified scrape_schedule for 19-20, 20-21, and 21-22 seasons individually 
# will make for easier processing of imputed features
schedule_1920 = pd.DataFrame(scrape_schedule('2019-10-02', '2020-03-11', preseason=False, not_over=False))
schedule_2021 = pd.DataFrame(scrape_schedule('2021-01-13', '2021-05-19', preseason=False, not_over=False))
schedule_2122 = pd.DataFrame(scrape_schedule('2021-10-12', '2022-05-01', preseason=False, not_over=False))
schedule_2223 = pd.DataFrame(scrape_schedule('2022-10-07', '2023-04-14', preseason=False, not_over=False))
schedule_1920

,game_id,game_type,season_id,date,home_score,away_score,Home_team,home_wins,home_losses,home_otl,away_wins,away_losses,away_otl,Away_team,status
0,2019020001,R,20192020,2019-10-02,5,3,TOR,1,0,0.0,0,1,0.0,OTT,Final
1,2019020002,R,20192020,2019-10-02,2,3,STL,0,0,1.0,1,0,0.0,WSH,Final
2,2019020003,R,20192020,2019-10-02,3,2,EDM,1,0,0.0,0,1,0.0,VAN,Final
3,2019020004,R,20192020,2019-10-02,4,1,VGK,1,0,0.0,0,1,0.0,S.J,Final
4,2019020005,R,20192020,2019-10-03,5,2,T.B,1,0,0.0,0,1,0.0,FLA,Final
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,2019021079,R,20192020,2020-03-11,6,2,CHI,32,30,8.0,29,36,5.0,S.J,Final
1079,2019021080,R,20192020,2020-03-11,2,4,EDM,37,25,9.0,37,28,6.0,WPG,Final
1080,2019020876,R,20192020,2020-03-11,2,4,ANA,29,33,9.0,42,19,10.0,STL,Final
1081,2019021081,R,20192020,2020-03-11,3,2,COL,42,20,8.0,37,28,5.0,NYR,Final


Bulk processing to conduct upon load:

- Convert the following fields accordingly
  - game_id, season_id to string
  - home_otl, away_otl to int
- Filter out non regular season games
  - game_type = R
- Add column to denote winner from perspective of home team
  - home_team = 1 denotes home team victory, 0 for loss
- Add game keys so schedule dfs can be joined with stats dfs
- Add running total column for number of games played so far that season for all teams
- Add running standings points total column for all teams
    - Win = 2 points
    - (Regulation) Loss = 0 points
    - Overtime loss = 1 point
- Add column for points percentage
    - pts_pct = actual points accumulated / potential max points 
    - potential max points = games played * 2

In [34]:
# makes sure all games are 'R' type for regular season
schedule_1920['game_type'].value_counts()
# assert schedule_2021.loc[schedule_2021['game_type'] == 'R']
# assert schedule_2122.loc[schedule_2122['game_type'] == 'R']
# assert schedule_2223.loc[schedule_2223['game_type'] == 'R']

R     1082
WA       1
Name: game_type, dtype: int64

In [35]:
schedule_1920.loc[schedule_1920['game_type'] == 'WA']

,game_id,game_type,season_id,date,home_score,away_score,Home_team,home_wins,home_losses,home_otl,away_wins,away_losses,away_otl,Away_team,status
767,2019120001,WA,20192020,2020-01-24,1,2,AMERICAN ALL-STARS,0,0,NaN,0,0,NaN,CANADIAN ALL-STARS,Final


In [36]:
def schedule_df_processing(schedule_df):
    # filter out non regular season games - should have 3262 for 19-20 through 21-22 seasons
    schedule_df = schedule_df.loc[schedule_df['game_type'] == 'R']
    # convert game_id and season_id to str. make sure they aren't used numerically and in case needed for filtering
    schedule_df['game_id'] = schedule_df['game_id'].astype(str) # don't need
    schedule_df['season_id'] = schedule_df['season_id'].astype(str)
    # convert overtime loss columns to int from float as can't be decimal
    schedule_df['home_otl'] = schedule_df['home_otl'].astype(int)
    schedule_df['away_otl'] = schedule_df['away_otl'].astype(int)
    # add column for denoting game winner - denote home team victory in binary, win = 1
    schedule_df['Home_Team_Won'] = np.where(schedule_df['home_score'] > schedule_df['away_score'], 1, 0)
    # add keys for merging stats dfs
    schedule_df['Home_Team_Key'] = schedule_df['Home_team'].astype(str)+'_'+schedule_df['date'].astype(str)
    schedule_df['Away_Team_Key'] = schedule_df['Away_team'].astype(str)+'_'+schedule_df['date'].astype(str)
    # add column containing running total games played so far that season
    schedule_df['home_gp'] = (schedule_df['home_wins'] + schedule_df['home_losses'] + schedule_df['home_otl']).astype(int)
    schedule_df['away_gp'] = (schedule_df['away_wins'] + schedule_df['away_losses'] + schedule_df['away_otl']).astype(int)                              
    # add column containing running standing points total for teams. wins = 2, losses = 0, otl = 1
    schedule_df['home_points'] = ((schedule_df['home_wins'] * 2) + schedule_df['home_otl']).astype(int)
    schedule_df['away_points'] = ((schedule_df['away_wins'] * 2) + schedule_df['away_otl']).astype(int)
    # add column for points percentage column
    schedule_df['home_pts_pct'] = schedule_df['home_points'] / (schedule_df['home_gp'] * 2)
    schedule_df['away_pts_pct'] = schedule_df['away_points'] / (schedule_df['away_gp'] * 2)

    return schedule_df

In [37]:
# call processing function on all schedule dfs
schedule_1920 = schedule_df_processing(schedule_1920)
schedule_2021 = schedule_df_processing(schedule_2021)
schedule_2122 = schedule_df_processing(schedule_2122)
schedule_2223 = schedule_df_processing(schedule_2223)

In [38]:
# check lengths to make sure the correct number of games are returned
# 2542, 1082, 868, 1312, 1312
# print(len(schedule_1819))
print(len(schedule_1920))
print(len(schedule_2021))
print(len(schedule_2122))
print(len(schedule_2223))

1082
868
1312
1312


In [51]:
schedule_1920_2123 = pd.concat([schedule_1920, schedule_2021, schedule_2122, schedule_2223], axis=0, ignore_index=True)
schedule_1920_2123

,game_id,game_type,season_id,date,home_score,away_score,Home_team,home_wins,home_losses,home_otl,away_wins,away_losses,away_otl,Away_team,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_gp,away_gp,home_points,away_points,home_pts_pct,away_pts_pct
0,2019020001,R,20192020,2019-10-02,5,3,TOR,1,0,0,0,1,0,OTT,Final,1,TOR_2019-10-02,OTT_2019-10-02,1,1,2,0,1.000000,0.000000
1,2019020002,R,20192020,2019-10-02,2,3,STL,0,0,1,1,0,0,WSH,Final,0,STL_2019-10-02,WSH_2019-10-02,1,1,1,2,0.500000,1.000000
2,2019020003,R,20192020,2019-10-02,3,2,EDM,1,0,0,0,1,0,VAN,Final,1,EDM_2019-10-02,VAN_2019-10-02,1,1,2,0,1.000000,0.000000
3,2019020004,R,20192020,2019-10-02,4,1,VGK,1,0,0,0,1,0,S.J,Final,1,VGK_2019-10-02,S.J_2019-10-02,1,1,2,0,1.000000,0.000000
4,2019020005,R,20192020,2019-10-03,5,2,T.B,1,0,0,0,1,0,FLA,Final,1,T.B_2019-10-03,FLA_2019-10-03,1,1,2,0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4569,2022021310,R,20222023,2023-04-13,3,5,ANA,23,47,12,47,25,10,L.A,Final,0,ANA_2023-04-13,L.A_2023-04-13,82,82,58,104,0.353659,0.634146
4570,2022021311,R,20222023,2023-04-13,4,5,ARI,28,40,14,38,37,7,VAN,Final,0,ARI_2023-04-13,VAN_2023-04-13,82,82,70,83,0.426829,0.506098
4571,2022021312,R,20222023,2023-04-13,1,3,SEA,46,28,8,51,22,9,VGK,Final,0,SEA_2023-04-13,VGK_2023-04-13,82,82,100,111,0.609756,0.676829
4572,2022020547,R,20222023,2023-04-14,2,5,CBJ,25,48,9,42,33,7,BUF,Final,0,CBJ_2023-04-14,BUF_2023-04-14,82,82,59,91,0.359756,0.554878


In [52]:
print(schedule_1920_2123.columns.to_list())

['game_id', 'game_type', 'season_id', 'date', 'home_score', 'away_score', 'Home_team', 'home_wins', 'home_losses', 'home_otl', 'away_wins', 'away_losses', 'away_otl', 'Away_team', 'status', 'Home_Team_Won', 'Home_Team_Key', 'Away_Team_Key', 'home_gp', 'away_gp', 'home_points', 'away_points', 'home_pts_pct', 'away_pts_pct']


In [62]:
sched_cols = ['season_id', 'date', 'Home_Team_Won', 'Home_Team_Key', 'Away_Team_Key','Home_team', 'Away_team','home_pts_pct', 'away_pts_pct']
schedule_1920_2123 = schedule_1920_2123[sched_cols]
schedule_1920_2123

,season_id,date,Home_Team_Won,Home_Team_Key,Away_Team_Key,Home_team,Away_team,home_pts_pct,away_pts_pct
0,20192020,2019-10-02,1,TOR_2019-10-02,OTT_2019-10-02,TOR,OTT,1.000000,0.000000
1,20192020,2019-10-02,0,STL_2019-10-02,WSH_2019-10-02,STL,WSH,0.500000,1.000000
2,20192020,2019-10-02,1,EDM_2019-10-02,VAN_2019-10-02,EDM,VAN,1.000000,0.000000
3,20192020,2019-10-02,1,VGK_2019-10-02,S.J_2019-10-02,VGK,S.J,1.000000,0.000000
4,20192020,2019-10-03,1,T.B_2019-10-03,FLA_2019-10-03,T.B,FLA,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...
4569,20222023,2023-04-13,0,ANA_2023-04-13,L.A_2023-04-13,ANA,L.A,0.353659,0.634146
4570,20222023,2023-04-13,0,ARI_2023-04-13,VAN_2023-04-13,ARI,VAN,0.426829,0.506098
4571,20222023,2023-04-13,0,SEA_2023-04-13,VGK_2023-04-13,SEA,VGK,0.609756,0.676829
4572,20222023,2023-04-14,0,CBJ_2023-04-14,BUF_2023-04-14,CBJ,BUF,0.359756,0.554878


In [63]:
modeling_df = schedule_1920_2123.merge(all_stats_df.add_prefix('home_'), 
                                       left_on = 'Home_Team_Key', right_on = 'home_Game_Key', how = 'left')
modeling_df = modeling_df.merge(all_stats_df.add_prefix('away_'), 
                                       left_on = 'Away_Team_Key', right_on = 'away_Game_Key', how = 'left')
modeling_df['date'] = pd.to_datetime(modeling_df['date'])
modeling_df = modeling_df.drop(columns=['Home_Team_Key', 'Away_Team_Key', 'home_Date', 
                                        'home_Game_Key', 'away_Date', 'away_Game_Key'])
modeling_df

,season_id,date,Home_Team_Won,Home_team,Away_team,home_pts_pct,away_pts_pct,home_Team,home_rolling_avg_FF%,home_rolling_avg_GF%,home_rolling_avg_xGF%,home_rolling_avg_HDCF%,home_rolling_avg_HDSH%,home_rolling_avg_HDSV%,home_rolling_avg_SH%,home_rolling_avg_SV%,home_rolling_avg_GF/60_pp,home_rolling_avg_xGF/60_pp,home_rolling_avg_GA/60_pk,home_rolling_avg_xGA/60_pk,away_Team,away_rolling_avg_FF%,away_rolling_avg_GF%,away_rolling_avg_xGF%,away_rolling_avg_HDCF%,away_rolling_avg_HDSH%,away_rolling_avg_HDSV%,away_rolling_avg_SH%,away_rolling_avg_SV%,away_rolling_avg_GF/60_pp,away_rolling_avg_xGF/60_pp,away_rolling_avg_GA/60_pk,away_rolling_avg_xGA/60_pk
0,20192020,2019-10-02,1,TOR,OTT,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20192020,2019-10-02,0,STL,WSH,0.500000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20192020,2019-10-02,1,EDM,VAN,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20192020,2019-10-02,1,VGK,S.J,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20192020,2019-10-03,1,T.B,FLA,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4569,20222023,2023-04-13,0,ANA,L.A,0.353659,0.634146,ANA,37.465,35.833,38.186,37.886,15.757,83.117,94.583889,917.601389,5.613,5.128,10.733,8.345,L.A,52.020,57.485,48.423,52.619,13.876,87.212,84.458485,908.473636,24.340,9.049,11.679,7.225
4570,20222023,2023-04-13,0,ARI,VAN,0.426829,0.506098,ARI,46.469,39.654,45.404,42.538,18.801,79.739,95.675263,921.328947,8.369,4.214,18.592,11.571,VAN,45.407,54.939,44.357,47.319,18.985,84.714,100.100541,894.535135,10.092,8.368,23.087,8.659
4571,20222023,2023-04-13,0,SEA,VGK,0.609756,0.676829,SEA,57.050,54.449,59.066,56.560,16.971,70.043,100.511538,896.874872,10.103,7.819,1.600,4.080,VGK,49.700,62.403,54.140,54.399,22.219,86.241,100.515588,921.617059,4.185,7.559,26.538,9.003
4572,20222023,2023-04-14,0,CBJ,BUF,0.359756,0.554878,CBJ,36.813,39.326,35.056,37.632,14.366,73.425,77.141316,901.840000,7.399,6.745,18.434,10.780,BUF,49.983,51.548,49.511,47.102,20.448,86.842,92.056154,894.443590,5.203,5.691,6.636,7.356


In [57]:
# def rolling_wins(df, rolling_games=10):
#     df['rolling_avg_home_pts_pct'] = df.groupby([df['date'].dt.year, 'Home_team'])['home_pts_pct'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
#     df['rolling_avg_away_pts_pct'] = df.groupby([df['date'].dt.year, 'Away_team'])['away_pts_pct'].transform(lambda x: x.rolling(rolling_games, rolling_games).mean().shift())
#     return df

In [58]:
# modeling_df = rolling_wins(modeling_df)
# modeling_df

,season_id,date,Home_Team_Won,Home_team,Away_team,home_pts_pct,away_pts_pct,home_Team,home_rolling_avg_FF%,home_rolling_avg_GF%,home_rolling_avg_xGF%,home_rolling_avg_HDCF%,home_rolling_avg_HDSH%,home_rolling_avg_HDSV%,home_rolling_avg_SH%,home_rolling_avg_SV%,home_rolling_avg_GF/60_pp,home_rolling_avg_xGF/60_pp,home_rolling_avg_GA/60_pk,home_rolling_avg_xGA/60_pk,away_Team,away_rolling_avg_FF%,away_rolling_avg_GF%,away_rolling_avg_xGF%,away_rolling_avg_HDCF%,away_rolling_avg_HDSH%,away_rolling_avg_HDSV%,away_rolling_avg_SH%,away_rolling_avg_SV%,away_rolling_avg_GF/60_pp,away_rolling_avg_xGF/60_pp,away_rolling_avg_GA/60_pk,away_rolling_avg_xGA/60_pk,rolling_avg_home_pts_pct,rolling_avg_away_pts_pct
0,20192020,2019-10-02,1,TOR,OTT,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20192020,2019-10-02,0,STL,WSH,0.500000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20192020,2019-10-02,1,EDM,VAN,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20192020,2019-10-02,1,VGK,S.J,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20192020,2019-10-03,1,T.B,FLA,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4569,20222023,2023-04-13,0,ANA,L.A,0.353659,0.634146,ANA,37.465,35.833,38.186,37.886,15.757,83.117,94.583889,917.601389,5.613,5.128,10.733,8.345,L.A,52.020,57.485,48.423,52.619,13.876,87.212,84.458485,908.473636,24.340,9.049,11.679,7.225,0.382755,0.629831
4570,20222023,2023-04-13,0,ARI,VAN,0.426829,0.506098,ARI,46.469,39.654,45.404,42.538,18.801,79.739,95.675263,921.328947,8.369,4.214,18.592,11.571,VAN,45.407,54.939,44.357,47.319,18.985,84.714,100.100541,894.535135,10.092,8.368,23.087,8.659,0.444204,0.480173
4571,20222023,2023-04-13,0,SEA,VGK,0.609756,0.676829,SEA,57.050,54.449,59.066,56.560,16.971,70.043,100.511538,896.874872,10.103,7.819,1.600,4.080,VGK,49.700,62.403,54.140,54.399,22.219,86.241,100.515588,921.617059,4.185,7.559,26.538,9.003,0.610896,0.663412
4572,20222023,2023-04-14,0,CBJ,BUF,0.359756,0.554878,CBJ,36.813,39.326,35.056,37.632,14.366,73.425,77.141316,901.840000,7.399,6.745,18.434,10.780,BUF,49.983,51.548,49.511,47.102,20.448,86.842,92.056154,894.443590,5.203,5.691,6.636,7.356,0.361610,0.537830


In [64]:
modeling_df= modeling_df[[
    'season_id',
    'date',
    'Home_team',
  'Away_team',
  'Home_Team_Won',
  'home_rolling_avg_FF%',
  'home_rolling_avg_GF%',
  'home_rolling_avg_xGF%',
  'home_rolling_avg_HDCF%',
  'home_rolling_avg_HDSH%',
  'home_rolling_avg_HDSV%',
  'home_rolling_avg_SH%',
  'home_rolling_avg_SV%',
  'home_rolling_avg_GF/60_pp',
  'home_rolling_avg_xGF/60_pp',
  'home_rolling_avg_GA/60_pk',
  'home_rolling_avg_xGA/60_pk',
  'away_rolling_avg_FF%',
  'away_rolling_avg_GF%',
  'away_rolling_avg_xGF%',
  'away_rolling_avg_HDCF%',
  'away_rolling_avg_HDSH%',
  'away_rolling_avg_HDSV%',
  'away_rolling_avg_SH%',
  'away_rolling_avg_SV%',
  'away_rolling_avg_GF/60_pp',
  'away_rolling_avg_xGF/60_pp',
  'away_rolling_avg_GA/60_pk',
  'away_rolling_avg_xGA/60_pk',
#   'rolling_avg_home_pts_pct',
#   'rolling_avg_away_pts_pct',
]]
modeling_df.dropna(inplace=True)
modeling_df

,season_id,date,Home_team,Away_team,Home_Team_Won,home_rolling_avg_FF%,home_rolling_avg_GF%,home_rolling_avg_xGF%,home_rolling_avg_HDCF%,home_rolling_avg_HDSH%,home_rolling_avg_HDSV%,home_rolling_avg_SH%,home_rolling_avg_SV%,home_rolling_avg_GF/60_pp,home_rolling_avg_xGF/60_pp,home_rolling_avg_GA/60_pk,home_rolling_avg_xGA/60_pk,away_rolling_avg_FF%,away_rolling_avg_GF%,away_rolling_avg_xGF%,away_rolling_avg_HDCF%,away_rolling_avg_HDSH%,away_rolling_avg_HDSV%,away_rolling_avg_SH%,away_rolling_avg_SV%,away_rolling_avg_GF/60_pp,away_rolling_avg_xGF/60_pp,away_rolling_avg_GA/60_pk,away_rolling_avg_xGA/60_pk
141,20192020,2019-10-22,CGY,WSH,0,51.530,49.292,52.171,48.052,6.476,85.008,70.570606,916.011515,6.027,4.839,2.492,6.428,54.229,49.310,55.439,52.822,13.942,77.652,97.903437,916.051875,55.926,17.439,5.719,4.657
151,20192020,2019-10-24,DAL,ANA,1,47.510,44.729,52.979,56.640,23.892,81.519,74.160645,938.801613,5.211,6.131,5.980,4.955,49.075,59.341,50.647,56.223,19.764,92.559,80.736774,912.332581,1.173,4.920,6.614,4.886
153,20192020,2019-10-24,EDM,WSH,1,48.525,41.521,47.858,44.778,18.509,82.631,77.374545,911.065152,13.210,8.734,4.144,7.600,53.307,51.114,54.277,53.132,16.438,75.486,97.903437,916.051875,54.906,17.127,4.801,4.281
155,20192020,2019-10-25,TOR,S.J,1,50.669,49.713,46.325,43.564,26.335,77.804,83.725313,919.072188,9.900,6.012,6.126,5.894,47.301,40.113,42.496,41.539,16.089,81.599,82.280000,875.965937,13.254,9.169,3.008,6.854
158,20192020,2019-10-25,DET,BUF,0,48.081,35.525,49.142,50.718,19.480,75.680,57.837188,892.922187,2.299,4.348,12.069,7.171,49.556,60.775,47.158,46.951,29.491,86.141,83.693750,916.584687,11.082,6.916,8.905,8.075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4569,20222023,2023-04-13,ANA,L.A,0,37.465,35.833,38.186,37.886,15.757,83.117,94.583889,917.601389,5.613,5.128,10.733,8.345,52.020,57.485,48.423,52.619,13.876,87.212,84.458485,908.473636,24.340,9.049,11.679,7.225
4570,20222023,2023-04-13,ARI,VAN,0,46.469,39.654,45.404,42.538,18.801,79.739,95.675263,921.328947,8.369,4.214,18.592,11.571,45.407,54.939,44.357,47.319,18.985,84.714,100.100541,894.535135,10.092,8.368,23.087,8.659
4571,20222023,2023-04-13,SEA,VGK,0,57.050,54.449,59.066,56.560,16.971,70.043,100.511538,896.874872,10.103,7.819,1.600,4.080,49.700,62.403,54.140,54.399,22.219,86.241,100.515588,921.617059,4.185,7.559,26.538,9.003
4572,20222023,2023-04-14,CBJ,BUF,0,36.813,39.326,35.056,37.632,14.366,73.425,77.141316,901.840000,7.399,6.745,18.434,10.780,49.983,51.548,49.511,47.102,20.448,86.842,92.056154,894.443590,5.203,5.691,6.636,7.356


In [65]:
modeling_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3728 entries, 141 to 4573
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   season_id                   3728 non-null   object        
 1   date                        3728 non-null   datetime64[ns]
 2   Home_team                   3728 non-null   object        
 3   Away_team                   3728 non-null   object        
 4   Home_Team_Won               3728 non-null   int64         
 5   home_rolling_avg_FF%        3728 non-null   float64       
 6   home_rolling_avg_GF%        3728 non-null   float64       
 7   home_rolling_avg_xGF%       3728 non-null   float64       
 8   home_rolling_avg_HDCF%      3728 non-null   float64       
 9   home_rolling_avg_HDSH%      3728 non-null   float64       
 10  home_rolling_avg_HDSV%      3728 non-null   float64       
 11  home_rolling_avg_SH%        3728 non-null   float64   

In [66]:
modeling_df.to_csv('modeling_final.csv', index=False)

In [ ]:
# modeling_df['Date'] = modeling_df['home_Date']
# modeling_df = modeling_df.drop(columns=['Home_Team_Key', 'Away_Team_Key', 'home_Date', 
#                                         'home_Game_Key', 'away_Date', 'away_Game_Key'])
# modeling_df

In [ ]:
modeling_df.info()